In [1]:
import cv2 as cv
import numpy as np
import keyboard
import time

import kalman_filter
import Controlling_thymio
import global_planning
import vision
import loc_avoid
from ekf import ExtendedKalmanFilter
from ekf import apply_kalman

In [2]:
# Global constants here

IMAGE_WIDTH = 640 # width of the video feed

IMAGE_HEIGHT = 480 # height of the video feed

CAMERA_FPS = 30 # fps of the video feed

KEYBOARD_INPUT = "enter" # keyboard input to start

CAMERA_REFRESH_TIME = 30 # number of frame thrown away to allow the camera to focus in the meantime

SKIPPED_FRAME = 3 # get one frame every X

ACCEPTABLE_MOVEMENT = 100

In [3]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
import Controlling_thymio

node = await client.wait_for_node()

await node.lock()
aw(node.lock())

await node.wait_for_variables({"acc"})

In [4]:
def check_angle(angle):
    if  isinstance(angle, np.ndarray):
        return float(angle[0])
    else:
        return float(angle)


In [ ]:
# 0 is laptop webcam, 1 is USB camera (!COMPUTER DEPENDENT!)
cap = cv.VideoCapture(0)

init = False # boolean set to True on keyboard input to start initialisation
is_init = False # boolean set to True once initialisation has succeeded
being_kidnapped = False # boolean set to True while the robot is being kidnapped
error_linear = [] # array of all linear errors
error_angle = [] # array of all angular errors
initial_turn = True # boolean set to True while the first orientation is not done
doing_local_nav = False # boolean set to True when we just finished doing local navigation

# variable initialized at False irrelevant of wether it actually is
robot_not_found = False # boolean set to True if robot not found for one frame
camera_hidden = False # boolean set to True if robot not found for multiple frames

# create a robot instance
robot_instance = Controlling_thymio.Robot()

# if unable to connect to the camera
if not (cap.isOpened()):
    print("Could not open video device")
else:
    while(True):
        # get the input frame by frame (shape (480,640,3))
        ret, frame = cap.read() 
        cap.set(cv.CAP_PROP_FPS, CAMERA_FPS) 
        frame = cv.resize(frame, (IMAGE_WIDTH,IMAGE_HEIGHT), interpolation=cv.INTER_CUBIC) 

        # wait until keyboard input to initialize vision
        if keyboard.is_pressed(KEYBOARD_INPUT):
            init = True

        # wait until keyboard input to initialize vision
        if keyboard.is_pressed("h"):
            if camera_hidden:
                camera_hidden = False
            else:
                camera_hidden = True

        # reinitialize the robot after a kidnapping
        if being_kidnapped: 
            if not Controlling_thymio.kidnapping(node):
                
                # to give time to the kidnapper to get out of the frame
                time.sleep(3)
                
                # reinitialize the starting position of the robot
                fop = vision.get_fop(frame, original_coordinates, new_coordinates)
                try:
                    start_x, start_y, robot_angle, width = vision.get_robot_position(fop)
                except:
                    break

                # reinitialize the global planning
                goal = (objective_x, objective_y)
                robot_instance.update_coordinates(start_x, start_y, robot_angle, width)
                path, obstacles_named= global_planning.global_plan(obstacles, robot_instance, goal)

                black_frame_remaining = 0
                initial_turn = True
                is_init = True
                being_kidnapped = False
        
        # initialize 
        if init == True:
            try:
                # vision: start, obstacles and goal coordinates
                original_coordinates, new_coordinates = vision.get_fop_coordinates(frame)
                fop = vision.get_fop(frame, original_coordinates, new_coordinates)
                #start_x, start_y, robot_angle, width = vision.get_robot_position(fop)

                start_x, start_y, robot_angle, width = vision.get_robot_position(fop)

                #print(start_x,start_y)
                obstacles = vision.get_obstacles(fop, width)
                objective_x, objective_y = vision.get_objective(fop)

                # global planning
                goal = (objective_x, objective_y)
                robot_instance.update_coordinates(start_x, start_y, robot_angle, width)
                path, obstacles_named = global_planning.global_plan(obstacles, robot_instance, goal)
                
                # kalman filter
                kalman = kalman_filter.KalmanFilter(np.array([[start_x], [start_y], [robot_angle]], dtype=object))

                left_wheel_speed = 0
                right_wheel_speed = 0

                being_kidnapped = False
                is_init = True
            except Exception as e:
                print("Initialisation failed: " + str(e))

            init = False

        if is_init:  
        ################################## MOTION CONTROL HERE ################################## 
            
            # get out of the loop if the robot is being kidnapped
            if Controlling_thymio.kidnapping(node):
                Controlling_thymio.set_speed(0, 0, aw, node)
                being_kidnapped = True
                is_init = False
                continue

            fop = vision.get_fop(frame, original_coordinates, new_coordinates)

            if camera_hidden:
                rows = fop.shape[0]
                columns = fop.shape[1]

                for i in range(rows):
                    for j in range(columns):
                        fop[i][j] = [0,0,0]

            try:
                center_x, center_y, robot_angle, width = vision.get_robot_position(fop)
                if abs((center_x + center_y) - (robot_instance.center_x + robot_instance.center_y)) > ACCEPTABLE_MOVEMENT:
                    raise
                
                #robot_instance.update_coordinates(center_x, center_y, robot_angle, width)
                robot_instance.update_coordinates((center_x + int(kalman.x[0]))/2, (center_y + int(kalman.x[1]))/2, (robot_angle + int(kalman.x[2]))/2, width)
            except:
                center_x = 0
                center_y = 0
                robot_angle = float(0)
            
            # vision able to send position
            if center_x != 0:
                 position = np.array([[center_x], [center_y], [check_angle(robot_angle)]], dtype=float)
                 kalman.update(position)
            else:
                robot_instance.update_coordinates(int(kalman.x[0]), int(kalman.x[1]), kalman.x[2], width)

            # kalman filter update
            
            [delta_x, delta_y, delta_angular] = kalman_filter.convert_input(robot_instance)
            new_speed = np.array([[delta_x], [delta_y], [delta_angular]], dtype=float)
            kalman.predict(new_speed)            

            start_point = np.array([obstacles_named[path[0]][0],obstacles_named[path[0]][1]])
            end_point = np.array([obstacles_named[path[1]][0],obstacles_named[path[1]][1]])
            
            # traget point after the actual end point
            try:
                next_point = np.array([obstacles_named[path[2]][0],obstacles_named[path[2]][1]])
            except:
                next_point = [0, 0]

            # proximity avoidance
            if loc_avoid.check_local_nav(node, robot_instance, aw):
                left_wheel_speed, right_wheel_speed = loc_avoid.local_avoidance(node, robot_instance, aw)
                initial_turn = True
                doing_local_nav = True     
            else:
                # reinitialize global planning from the position reached in local navigation
                if doing_local_nav:
                    path, obstacles_named = global_planning.global_plan(obstacles, robot_instance, goal)
                    doing_local_nav = False
                left_wheel_speed, right_wheel_speed, error_linear, error_angle, initial_turn, path = Controlling_thymio.compute_wheel_speed(initial_turn, start_point, end_point, next_point, error_linear, error_angle, robot_instance, path, aw, node)   

            robot_instance.update_speed(left_wheel_speed, right_wheel_speed)

            # draw everything
            cv.circle(fop, (robot_instance.center_x,robot_instance.center_y), radius=10, color=(0, 255, 0), thickness=-1)
            cv.circle(fop, (objective_x, objective_y), radius=10, color=(255, 0, 0), thickness=-1)
            cv.circle(fop, (start_x, start_y), radius=10, color=(0, 0, 255), thickness=-1)
            for obstacle in obstacles:
                for vertice in obstacle:
                    cv.circle(fop, (vertice[0],vertice[1]), radius=10, color=(0, 255, 0), thickness=-1)
            for i in range(len(path[:-1])):
                cv.line(fop,(obstacles_named[path[i]][0],obstacles_named[path[i]][1]),(obstacles_named[path[i+1]][0],obstacles_named[path[i+1]][1]),(255,0,0),5)
    
        #########################################################################################
        else:
            # to output the raw feed
            fop = frame

        cv.imshow('Raw',frame)
        cv.imshow('Processed',fop)

        await client.sleep(0.001)
        
        # Close all windows and get out of the loop if ESC is pressed
        if cv.waitKey(1) == 27:
            cv.destroyAllWindows()
            break

Initialisation failed: division by zero
   
Kalman:  [[48]
 [300]
 [array([0.9151007])]]
vision:  48 300 [0.9151007]
delta angle:  0.0
init turn


C:\Users\zalex\AppData\Local\Temp\ipykernel_1184\2335455374.py:134: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  robot_instance.update_coordinates(int(kalman.x[0]), int(kalman.x[1]), kalman.x[2], width)


delta angle:  0.008518621546489545
delta angle:  0.008518621546489545
   
Kalman:  [[array([50.41721052])]
 [array([303.17022335])]
 [array([0.93213794])]]
vision:  49 299 [0.82537685]
delta angle:  -0.0025925954969689365
   
Kalman:  [[array([50.70453063])]
 [array([301.43211084])]
 [array([0.84650894])]]
vision:  50 297 [0.66104317]
delta angle:  0.017408286653115657
delta angle:  0.035192449093082244
   
Kalman:  [[array([53.37645274])]
 [array([301.24263565])]
 [array([0.78041158])]]
vision:  55 295 [0.96525166]
delta angle:  0.015185768837343554
delta angle:  0.04631285043953316
   
Kalman:  [[array([56.84710508])]
 [array([299.90951152])]
 [array([0.97597004])]]
vision:  55 293 [0.08314123]
delta angle:  0.024076400079997933
delta angle:  0.05224598779632137
   
Kalman:  [[array([59.1060981])]
 [array([295.6073187])]
 [array([0.39913153])]]
vision:  60 292 [0.5404195]
delta angle:  0.0422347773356894
delta angle:  0.05558417328091747
delta angle:  0.047054399101554414
   
Kalman: